In [1]:
from json import load
import spacy
import nltk
import string
from nltk.corpus import stopwords
from string import punctuation
from spacy.matcher import PhraseMatcher


In [2]:
nlp = spacy.load('ru_core_news_sm')



In [3]:
with open('../../Data/Events/Leader_events.json', 'r', encoding='utf-8') as js:
    discription_of_event = load(js)
discription_of_event


{'120-й Игровой IT-субботник': {'название': '120-й Игровой IT-субботник',
  'описание': 'Встретимся на Игровом IT-субботнике!\nВ роли спикеров выступят:\n- Антон Петроченко с докладом «Как сделать игровой движок без математики, и почему так лучше не надо?».\n- Александр Гунгер с докладом «Свой игровой движок: велосипед или необходимость?».\n- Алексей Бондаренко с докладом «Программная сторона красивостей».',
  'train': {'название': {'value': 0, 'theme': 'Игровой'},
   'описание': {'type_of_event': 'доклад',
    'kcm': [1, 0, 0],
    'objects': ['«Свой игровой движок: велосипед или необходимость?»',
     '«Программная сторона красивостей»',
     '«Как сделать игровой движок без математики, и почему так лучше не надо?»']}}},
 'Рабочая встреча Омского Клуба мышления (весенний цикл).': {'название': 'Рабочая встреча Омского Клуба мышления (весенний цикл).',
  'описание': 'В форме представлений типов мышления, обсуждение их особенностей, применительности к реальным задачам, сравнительные хар

In [4]:
text = nlp(list(discription_of_event.values())[0]['описание'])

In [5]:
list(filter(lambda a: a.pos_ == 'VERB', text))


[Встретимся, выступят, сделать]

In [6]:
text2 = nlp(' '.join(list(map(lambda a: a.lower_, filter(
    lambda a: a.lower_ not in nlp.Defaults.stop_words and a.lower_ not in string.punctuation+'\n\t»«', text)))))


In [16]:
spacy.displacy.parse_deps(text2)['arcs']


[{'start': 0, 'end': 1, 'label': 'amod', 'dir': 'left'},
 {'start': 1, 'end': 6, 'label': 'obl', 'dir': 'left'},
 {'start': 1, 'end': 2, 'label': 'nmod', 'dir': 'right'},
 {'start': 1, 'end': 3, 'label': 'nmod', 'dir': 'right'},
 {'start': 3, 'end': 4, 'label': 'nmod', 'dir': 'right'},
 {'start': 4, 'end': 5, 'label': 'nmod', 'dir': 'right'},
 {'start': 6, 'end': 7, 'label': 'nsubj', 'dir': 'right'},
 {'start': 7, 'end': 8, 'label': 'flat:name', 'dir': 'right'},
 {'start': 6, 'end': 9, 'label': 'obl', 'dir': 'right'},
 {'start': 9, 'end': 10, 'label': 'nmod', 'dir': 'right'},
 {'start': 11, 'end': 12, 'label': 'amod', 'dir': 'left'},
 {'start': 10, 'end': 12, 'label': 'obj', 'dir': 'right'},
 {'start': 12, 'end': 13, 'label': 'nmod', 'dir': 'right'},
 {'start': 10, 'end': 14, 'label': 'advmod', 'dir': 'right'},
 {'start': 14, 'end': 15, 'label': 'appos', 'dir': 'right'},
 {'start': 15, 'end': 16, 'label': 'flat:name', 'dir': 'right'},
 {'start': 10, 'end': 17, 'label': 'obj', 'dir': 'r

In [8]:
spacy.displacy.render(text2)


In [9]:
list(map(lambda a: tuple([text2[a['start']],
     text2[a['end']]]), spacy.displacy.parse_deps(text2)["arcs"]))


[(встретимся, игровом),
 (игровом, выступят),
 (игровом, it),
 (игровом, субботнике),
 (субботнике, роли),
 (роли, спикеров),
 (выступят, антон),
 (антон, петроченко),
 (выступят, докладом),
 (докладом, сделать),
 (игровой, движок),
 (сделать, движок),
 (движок, математики),
 (сделать, лучше),
 (лучше, александр),
 (александр, гунгер),
 (сделать, докладом),
 (игровой, движок),
 (докладом, движок),
 (докладом, велосипед),
 (сделать, необходимость),
 (необходимость, алексей),
 (алексей, бондаренко),
 (сделать, докладом),
 (программная, сторона),
 (выступят, сторона),
 (сторона, красивостей)]

In [10]:
# matcher = PhraseMatcher(nlp.vocab)
# matcher.add("Встретимся", [nlp("Barack Obama")])
# matcher(text2)


def on_match(matcher, doc, id, matches):
    print('Matched!', matches)

matcher = PhraseMatcher(nlp.vocab)
matcher.add("OBAMA", [nlp("Barack Obama")], on_match=on_match)
matcher.add("HEALTH", [nlp("health care reform"), 
                       nlp("healthcare reform"), 
                       nlp("healthcare reforms")], on_match=on_match)
doc = nlp("Barack Obama urges Congress to find courage to defend his healthcare ref")
matches = matcher(doc)

Matched! [(7732777389095836264, 0, 2)]


In [11]:
text2.ents

(антон петроченко, александр гунгер, алексей бондаренко)

In [12]:
list(filter(lambda a: a.tag_ == 'VERB', text)), list(filter(lambda a: a.tag_ == 'VERB', text2))


([Встретимся, выступят, сделать], [встретимся, выступят, сделать, велосипед])

In [13]:
text

Встретимся на Игровом IT-субботнике!
В роли спикеров выступят:
- Антон Петроченко с докладом «Как сделать игровой движок без математики, и почему так лучше не надо?».
- Александр Гунгер с докладом «Свой игровой движок: велосипед или необходимость?».
- Алексей Бондаренко с докладом «Программная сторона красивостей».

In [14]:
nlp('Дискуссия')[0].lemma_


'дискуссия'

Программа:ы частичное отображение из мн-тва всех возможных состояний в себя.
Синхранизация: задать фцию которая не раскладывается в композицию двух других.
чарльз энтони хоур
Полиморфизм $p_1 \rightarrow p$

